In [ ]:
import tableauserverclient as TSC
from tableauhyperapi import HyperProcess, Telemetry, \
    Connection, CreateMode, \
    NOT_NULLABLE, NULLABLE, SqlType, TableDefinition, \
    Inserter, \
    escape_name, escape_string_literal, \
    HyperException,TableName

from pathlib import Path
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils.querying import get_projects_dataframe
import pandas as pd

In [ ]:
# Login to Tableau Online and open a session
tableau_server_config = {
        'my_env': {
                'server': 'https://10ax.online.tableau.com',
                'api_version': '3.12',
                'personal_access_token_name':'vivek',
                'personal_access_token_secret':'',
                'site_name': '',
                'site_url': ''
        }
}
conn = TableauServerConnection(tableau_server_config, env='my_env')
conn.sign_in()

In [ ]:
with HyperProcess(telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU ) as hyper:

#  Connect to an existing .hyper file (CreateMode.NONE)
    with Connection(endpoint=hyper.endpoint, database='Covid.hyper') as connection:
        table_names = connection.catalog.get_table_names(schema="Extract")
         #connection.query_method(query="query_string")
        for table in table_names:
                table_definition = connection.catalog.get_table_definition(name=table)
                print(f"Table {table.name} has qualified name: {table}")
                # Print Column Type
                for column in table_definition.columns:
                    print(f"Column {column.name} has type={column.type} and nullability={column.nullability}")
                print("")
        # Print all rows from the "Extract"."Extract" table.
        table_name = TableName("Extract", "Extract")
        print(f"These are all rows in the table {table_name}:")
        # Get data from hyper file
        rows_in_table = connection.execute_list_query(query=f"SELECT * FROM {table_name}")
        # Create Data frame and use for further Analysis
        df=pd.DataFrame(rows_in_table)
        print(rows_in_table)
        print("The connection to the Hyper file has been closed.")
    print("The Hyper process has been shut down.")
conn.sign_out()

In [ ]:
df.head()
df.sort_values(4,ascending=False)

In [ ]:
# Assign column names
for col in df.columns:
    print(col)
df.columns=["S. No.","Name of State UT","Active Cases","Cured_Discharged_Migrated","Deaths","Total Confirmed cases"]